In [4]:
# Import required modules
# For NewsAPI
import requests
import numpy as np
import pandas as pd
import csv

# For Individual Feature Extraction 
from sklearn.feature_extraction.text import CountVectorizer
import nltk
import re
from collections import Counter

# For Relational Feature Extraction
from statistics import mean
import math

# To shuffle the dataset
from sklearn.utils import shuffle

# News Scrap API

<h4>This program scraps a news about specific topic from newsapi.org API.</h4>
To change topic (or select topic), change q = '' part
To get specific news domain, use domains = '<news domain url>&'
Use page=#& to get more results

Data includes: Author, title, description, url, content, publishing date, source ID

In [5]:
def news_api(topic, number, source, date, reliability):
    # List creation
    source_id = []
    author = []
    title = []
    description = []
    url = []
    content = []
    pub_date = []
    rel = []

    #set the reliability list by user input
    if reliability is 1:
        rel = np.ones(number)
    elif reliability is 0:
        rel = np.zeros(number)
    
    # Read 100 news articles about coronavirus (20 articles per each page, 5 pages) and parse each data into corresponding lists
    for i in range(1, int((number/20)+1)):

        # News extraction
        news_url = ('https://newsapi.org/v2/everything?'
                f'domains={source}&'
                f'q={topic}&'
               f'page={i}&'
               f'from={date}&'
               'sortBy=popularity&'
               'apiKey=ddb22f3ef2b54abe8af228db83421424')

        news = requests.get(news_url)
    
        for elements in news.json()['articles']:
            source_id.append(elements['source']['id'])
            author.append(elements['author'])
            title.append(elements['title'])
            description.append(elements['description'])
            url.append(elements['url'])
            content.append(elements['content'])
            pub_date.append(elements['publishedAt'])

            # *For CNN Only: check if the content is video and data does not related to the article
            #if (elements['content'] == 'None' or elements['content'] == "Chat with us in Facebook Messenger. Find out what's happening in the world as it unfolds."):
            #    pass
                #print(elements['description'])
            #else:
                #print (elements['content'])
            #    pass

    # Test for parsed data    
    #print(author)
    news_info = pd.DataFrame()
    news_info['source_id'] = source_id
    news_info['author'] = author
    news_info['title'] = title
    news_info['description'] = description
    news_info['url'] = url
    news_info['content'] = content
    news_info['published_date'] = pub_date
    news_info['label'] = rel
    return news_info
    

    
news = news_api('coronavirus', 100, 'cnn.com', '2020-01-27', 1)
text = news_api('coronavirus', 100, 'theverge.com, foxnews.com', '2020-01-27', 0)

#print(news['description'].head(50))
#print(news['description'].tail(50))
#if news['description'] is "":
print(len(news['description'][65]))

201


In [6]:
'''
This program is only for feature extraction from given data.

At the beginning, the program grab the news contents (if content is null, use description. If both are
null, drop that news.) from the given dataset and extract the features from the news contents.
Then, it saves extracted data and features in a file.

Features used in this program are:
1. Number of Characters
2. Number of Words
3. Number of Verbs
4. Number of Nouns
5. Number of Sentence
6. Average Number of Words per Sentence
7. Average Number of Characters in Words
8. Number of Question Marks
9. Percentage of Subjective Verbs
10. Percentage of Passive Voice
11. Percentage of Positive Words
12. Percentage of Negative Words
13. Lexical Diversity: Unique Words or Terms
14. Typographical Error Ration: Misspelled Words
15. Causation Terms

How to use: First, place positive-words.txt and negative-words.txt files in word_sentiment folder and put
            .csv dataset in liar_dataset folder in datasets folder and put the name of the .csv file
            in argument space of setup_data function. Next, use feature_extraction function with the result of
            setup_data function to extract the features and save the data in the desired file.

Date: Jan 28th, 2020
Programmed by: Sasung Kim
'''

'\nThis program is only for feature extraction from given data.\n\nAt the beginning, the program grab the news contents (if content is null, use description. If both are\nnull, drop that news.) from the given dataset and extract the features from the news contents.\nThen, it saves extracted data and features in a file.\n\nFeatures used in this program are:\n1. Number of Characters\n2. Number of Words\n3. Number of Verbs\n4. Number of Nouns\n5. Number of Sentence\n6. Average Number of Words per Sentence\n7. Average Number of Characters in Words\n8. Number of Question Marks\n9. Percentage of Subjective Verbs\n10. Percentage of Passive Voice\n11. Percentage of Positive Words\n12. Percentage of Negative Words\n13. Lexical Diversity: Unique Words or Terms\n14. Typographical Error Ration: Misspelled Words\n15. Causation Terms\n\nHow to use: First, place positive-words.txt and negative-words.txt files in word_sentiment folder and put\n            .csv dataset in liar_dataset folder in dataset

In [7]:
#########################################################################################################
# Create the lists of the words that used for feature extraction

positive_list = list()
negative_list = list()

# Gather the positive and negative word lists from the text file
with open('./word_sentiment/positive-words.txt') as p:
    for line in p:
        val = line.split()
        for ele in val:
            positive_list.append(ele)

with open('./word_sentiment/negative-words.txt') as n:
    for line in n:
        val = line.split()
        for ele in val:
            negative_list.append(ele)

subjective_list = list(['am', 'are', 'is', 'was', 'were', 'be', 'been'])
causation_list = list(['led to', 'because', 'cause', 'caused', 'reason', 'explanation', 'so'])

exclusive_list = list(['except', 'else', 'besides', 'without', 'exclude', 'other than'])
generalizing_list = list(['all', 'none', 'most', 'many', 'always', 'everyone','never',
                          'some','usually','few','seldom','generally','general','overall'])
pronoun_1st_list = list(['I','we'])
pronoun_2nd3rd_list = list(['you','your','yours','he','she','it','him','her','his','her','its','hers','They','them','theirs','their'])

SPronoun_list = list(['I', 'mine','my','me'])
GPronoun_list = list(['we','ours','our','us'])

In [8]:

#########################################################################################################

# From the dataset, isolate the required data (statement and label in this program) and return as pandas Series
# If the user needs the required data as a file, uncomment def with new_file as argument and comment the other one

#def setup_data(file_name, new_file):
def setup_data(news_info):
    
    # Check for data reading
    #print(news_info.head(5))
    
    # Check for the raw data from original dataset
    # print (list(data.columns.values))
    # print(data.tail(1))

    data = pd.DataFrame()
    news_contents = []
    label_list = []
    
    for elements in range(len(news_info)):
        description = news_info.loc[elements, 'description']
        label = news_info.loc[elements, 'label']
        
        # uncomment and indent the news_contents.append(description) line if both contents and descriptions are used instead of  description only
        
        #content = data.loc[elements, 'content']
        #if not pd.isna(content) and content is "Chat with us in Facebook Messenger. Find out what's happening in the world as it unfolds.":
        #    news_contents.append(content)
        #else:
        news_contents.append(description)
    
        label_list.append(label)

    
    data['description'] = news_contents
    data['label'] = label_list
    
    # drop the null values (if both content and description are null)
    data = data.dropna(how = 'any')
    for i in range(len(data)):
        try:
            if (len(data['description'][i]) == 0):
                data = data.drop(i)
            else:
                pass
        except:
            pass
    return data

# Data Extraction
# From the dataset file, gather the news contents and labels only

text_data = setup_data(text)
news_data = setup_data(news)

print(text_data.head(5))
print(news_data.head(5))

                                         description  label
0  Coronaviruses caused SARS, MERS, and the curre...    0.0
1  The coronavirus — which originated in Wuhan, C...    0.0
2  The tech sector is feeling the impact of gover...    0.0
3  Recommendations for blunting the spread of inf...    0.0
4    The State Department issued the travel advisory    0.0
                                         description  label
0  Asians and Asian Americans are facing increase...    1.0
1  Future cruisegoers are expressing concern over...    1.0
2  US stock futures dropped Sunday evening as the...    1.0
3  The novel coronavirus-related shock to US mark...    1.0
4  Apple is temporarily closing all of its stores...    1.0


In [9]:
# Input: String ('str')
# Description: Tockenize and tag input with nltk universal tag
# Return: tags of each tockens - String ('tagged')
# Tockenize and tag input String with nltk universal tag and return the tags for each words in the String

def tagging_univ(str):
    text = nltk.word_tokenize(str)
    tagged = nltk.pos_tag(text, tagset='universal')
    return tagged

# Input: String('str')
# Description: Tockenize and tag input with nltk non-universal tag
# Return: tags of each tockens - String ('tagged')

def tagging_nuniv(str):
    text = nltk.word_tokenize(str)
    tagged = nltk.pos_tag(text)
    return tagged

# Input: String('str')
# Description: Count the number of characters in input
# Return: Character count - int ('count')

def count_char(str):
    no_space = str.replace(" ", "")
    count = len(no_space)
    return count

# Input: String('str')
# Description: Count the number of words in input
# Return: Word count - int ('count')

def count_word(str):
    count = len(str.split())
    return count

# Input: String ('str')
# Description: Count the number of sentences by counting number of period(.)
# Return: Sentence count - int ('sentence')

def count_sent(str):
    sentence = len(str.split('.'))
    return sentence

# Input: String ('states')
# Description: Count the number of characters in each word in input and average the number of characters per word
# Return: Average number of characters: float ('avg')

def count_char_per_word(states):
    word = []
    word.append(states.split())
    char_per_word = list()
    for elements in word:
        for char in elements:
            c_in_w_count = len(char)
            char_per_word.append(c_in_w_count)
    # char_per_word_list.append()
    avg = sum(char_per_word) / len(char_per_word)
    char_per_word.clear()
    return avg

"""
Change comments
"""

# Input: String ('tagged'), String ('tag'), count ('int')
# Description: Count the number of common words between input String and input list
# Return: Count of common words - int ('int')
# Count and return the number of common words between input String ('tagged') and input list

def check_tag(check, tag, count):
    if check == tag:
        count += 1
    return count

# Input: String ('tagged'), list ('list'), int ('int')
# Description: Count the number of common words between input String and input list
# Return: Count of common words - int ('int')
# Count and return the number of common words between input String ('tagged') and input list

def check_common(state, list, count):
    #for ele in list:
    #    count = tagged.count(ele)
    #return count
    for elements in list:
        for words in state.split():
            if words == elements:
                count += 1
    return count

# Input: String ('word'), String ('tag'), list ('subjective_list'), int ('sent_count')
# Description: Count the number of passive voice and avarge it from number of sentence (sent_count)
# Return: Average of passive voiced sentences - float ('result')

def count_passive(word, tag, subjective_list, sent_count):
    percent_sub = 0
    counter = 0
    for ele in subjective_list:
        if (word.count(ele) > 0 and tag == "VBN"):
            counter += 1
    result = counter / sent_count * 100
    return result

# Input: String ('states')
# Description: Count the words that introduced only once in input
# Return: Count of unique words - int ('unique_count')

def count_unique(states):
    words = states.split(' ')
    c = Counter(words)
    unique = [w for w in words if c[w] == 1]
    unique_counter = len(unique)
    return unique_counter

# Calvin
def avg_char(states):
    word.append(states.split())
    char_per_word = list()
    for elements in word:
        for char in elements:
            c_in_w_count = len(char)
            char_per_word.append(c_in_w_count)
    word.clear()
    avg = sum(char_per_word)/len(char_per_word)
    char_per_word.clear()
    return avg


# Input: panda Series ('data'), String ('save_file_name')
# Description: First, this function creates lists for each features and extract the features using statements in dataset and
#              above functions. Next, it creates a large pandas Series that consist of news contents, labels and extracted
#              features. Finally, it save the final pandas Series in a file to make easier to examine the result (do not need
#              to rerun the program or change the code to check raw data)
# Return: Pandas Series consist of news contents, labels and extracted features count - pandas Series ('new')

def feature_extract(data):
    # define the news contents and labels from the dataset
    state = data['description']
    label = data['label']
    
    # Create a dataframe that will store every feature values
    ind_feature = pd.DataFrame()

    # create lists for storing the counters
    char_count_list = list()
    word_count_list = list()
    verb_count_list = list()
    noun_count_list = list()
    sent_count_list = list()
    words_per_sent_list = list()
    char_per_word_list = list()
    quest_count_list = list()
    sub_count_list = list()
    pass_count_list = list()
    pos_count_list = list()
    neg_count_list = list()
    unique_count_list = list()
    typo_count_list = list()
    cause_count_list = list()
    
    # Calvin
    gene_count_list = list()
    num_count_list = list()
    pron_1st_count_list = list()
    pron_2nd3rd_count_list = list()
    exclu_count_list = list()
    
    # Andrew
    exclam_list = list()
    lex_list = list()
    singlulars_list = list()
    group_list = list()
    other_list = list()


    word = list()

    #print(type(state))
    # loop for checking each new contents in dataset
    for states in state:

        #print(states)
        # reset the counters for each news contents
        w_in_s_count = 0
        c_in_w_count = 0
        verb_count = 0
        noun_count = 0
        sub_count = 0
        pos_count = 0
        neg_count = 0
        percent_pos = 0
        percent_neg = 0
        unique_count = 0
        sent_counts = 0
        typo_count = 0
        cause_count = 0
        gene_count = 0
        pron_1st_count = 0
        pron_count = 0
        exclu_count = 0
        SelfP = 0
        count_prongroup = 0
        group = 0
        other = 0
        num= 0
        
        # Tockenization and tagging with nltk universal and non-universal tag systems (tagged = universal, tagged_nu = non-universal)
        tagged = tagging_univ(states)
        tagged_nu = tagging_nuniv(states)

        # Check the tags of each news contents
        # print(tagged)
        # print(tagged_nu)

        # Extract the features and append the results in the list. Commented lines with print() functions are for testing

        # 1. Number of Characters
        char_count = count_char(states)
        char_count_list.append(char_count)

        # 2. Nubmer of Words
        word_count = count_word(states)
        word_count_list.append(word_count)

        # 3. Number of Verbs
        for tag in tagged:
            verb_count = check_tag(tag[1], 'VERB', verb_count)
        verb_count_list.append(verb_count)

        # 4. Number of Nouns
        for tag in tagged:
            noun_count = check_tag(tag[1], 'NOUN', noun_count)
        noun_count_list.append(noun_count)
        # print(noun_count)

        # 5. Number of Sentence
        sent_count = count_sent(states)
        sent_count_list.append(sent_count)
        # print(sent_count)

        # 6. Average number of words per sentence
        sent = [len(l.split()) for l in re.split(r'[?!.]', states) if l.strip()]
        w_in_s_count = (sum(sent) / len(sent))
        words_per_sent_list.append(w_in_s_count)
        # print(w_in_s_count)

        # 7. Average number of characters per word
        c_in_w_count = count_char_per_word(states)
        char_per_word_list.append(c_in_w_count)
        # print(c_in_w_count)

        # 8. Number of question marks
        quest_count = states.count("?")
        quest_count_list.append(quest_count)

        # 9. Percentage of subjective verbs - am/are/is/etc
        sub_count = check_common(states, subjective_list, sub_count)
        if (verb_count > 0):
            percent_sub = sub_count / verb_count * 100
        sub_count_list.append(percent_sub)

        # 10. Percentage of passive voice - am/are/is && past participate
        for tag in tagged_nu:
            passive_percent = count_passive(tag[0], tag[1], subjective_list, sent_count)
        pass_count_list.append(passive_percent)

        # 11. Percentage of positive words
        for tag in tagged:
            pos_count = check_common(tag[0], positive_list, pos_count)
            percent_pos = pos_count / word_count * 100
        pos_count_list.append(percent_pos)

        # 12. Percentage of negative words
        for tag in tagged:
            neg_count = check_common(tag[0], negative_list, neg_count)
            percent_neg = neg_count / word_count * 100
        neg_count_list.append(percent_neg)

        # 13. Lexical diversity: unique words or terms
        unique_count = count_unique(states)
        unique_count_list.append(unique_count)

        # 14. Typographical error ratio: misspelled words
        for tag in tagged:
            typo_count = check_tag(tag[1], 'X', typo_count)
        typo_count_list.append(typo_count)

        # 15. Causation terms
        cause_count = check_common(states, causation_list, cause_count)
        cause_count_list.append(cause_count)

        # 16. Percentage of generalizing terms
        gene_count = check_common(states, generalizing_list, gene_count)
        gene_count_list.append(gene_count)
        
        # 17. Percentage of numbers and quantifiers
        for tag in tagged:
            num = check_tag(tag[1], 'NUM', num)
        num_count_list.append(num)

        # 18. 1st person pronouns
        pron_1st_count = check_common(states, pronoun_1st_list, pron_1st_count)
        pron_1st_count_list.append(pron_1st_count)
   
        # 19. 2nd and 3rd person pronouns
        pron_count = check_common(states, pronoun_2nd3rd_list, pron_count)
        pron_2nd3rd_count_list.append(pron_count)

        # 20. Exclusive terms
        exclu_count = check_common(states, exclusive_list, exclu_count)
        exclu_count_list.append(exclu_count)
        
        # 21. # of exclamation marks
        exclam_count = states.count("!")
        exclam_list.append(exclam_count)
        
        # 22. lexical 
        unique_count = count_unique(states)
        lex_list.append(unique_count)
        
        # 23. singlular pronouns (1st person)
        SelfP = check_common(states, SPronoun_list, SelfP)
        singlulars_list.append(SelfP)
        
        # 24. Group ref pronouns (1st person)
        group = check_common(states, GPronoun_list, group)
        group_list.append(group)
        
        # 25. 2nd 3rd pronouns
        other = check_common(states, pronoun_2nd3rd_list, other)
        other_list.append(other)
        

    ind_feature['Statement'] = state
    ind_feature['Label'] = label
    ind_feature['# of Characters'] = char_count_list
    ind_feature['# of Words'] = word_count_list
    ind_feature['# of Verbs'] = verb_count_list
    ind_feature['# of Noun'] = noun_count_list
    ind_feature['# of Sentence'] = sent_count_list
    ind_feature['Average # of Words per Sentence'] = words_per_sent_list
    ind_feature['Average # of Characters per Words'] = char_per_word_list
    ind_feature['# of Question Marks'] = quest_count_list
    ind_feature['% of Subjective Verbs'] = sub_count_list
    ind_feature['% of Passive Voice'] = pass_count_list
    ind_feature['% of Positive Words'] = pos_count_list
    ind_feature['% of Negative Words'] = neg_count_list
    ind_feature['# of Unique Wrods/Terms'] = unique_count_list
    ind_feature['# of Misspelled Words'] = typo_count_list
    ind_feature['# of Causation Terms'] = cause_count_list
    ind_feature['% of generalizing terms'] = gene_count_list
    ind_feature['% of # and quantifiers'] = num_count_list
    ind_feature['1st person pronouns'] = pron_1st_count_list
    ind_feature['2nd and 3rd person pronouns'] = pron_2nd3rd_count_list
    ind_feature['Exclusive term'] = exclu_count_list
    ind_feature['# of exclamation marks'] = exclam_list
    ind_feature['Lexical'] = lex_list
    ind_feature['Singular pronouns(1st person)'] = singlulars_list
    ind_feature['Group ref pronouns(1st person)'] = group_list
    ind_feature['2nd 3rd pronouns'] = other_list

    return ind_feature


########################################################################################################################


# Check for shape and contents of isolated data
# print(train_data.shape)
# print(test_data.shape)
# print (valid_data.shape)
# print (train_data.head(5))
# print (test_data.head(5))
# print (valid_data.head(5))

# Feature Extraction

text_data_FE = feature_extract(text_data)
news_data_FE = feature_extract(news_data)

# Checking data

print(text_data_FE.head(5))
print(news_data_FE.head(5))


                                           Statement  Label  # of Characters  \
0  Coronaviruses caused SARS, MERS, and the curre...    0.0              150   
1  The coronavirus — which originated in Wuhan, C...    0.0              220   
2  The tech sector is feeling the impact of gover...    0.0              115   
3  Recommendations for blunting the spread of inf...    0.0              189   
4    The State Department issued the travel advisory    0.0               41   

   # of Words  # of Verbs  # of Noun  # of Sentence  \
0          28           7          9              3   
1          42           6         15              2   
2          19           3          9              2   
3          31           5         11              3   
4           7           1          4              1   

   Average # of Words per Sentence  Average # of Characters per Words  \
0                             14.0                           5.357143   
1                             21.0        

In [10]:
"""
This program extract the multi-sourced features from two datasets (one is unreliable dataset that text that is evaluated, one for reliable dataset as reference news of evaluation).

Restrictions: Two provided datasets must have the same number of data and have same feature and name. For better use, it is recommended to use the news/texts that have same topics.

How to use: Place the datasets that consists of statements and feature values in a folder and put the name or path of the datasets in news1 and news2 arguments of multi_source_FE() definition.
            news1 should be the dataset of referencing articles, and news2 should be the dataset of testing texts

Date: Feb 2, 2020
Programmed by: Sasung Kim

"""

'\nThis program extract the multi-sourced features from two datasets (one is unreliable dataset that text that is evaluated, one for reliable dataset as reference news of evaluation).\n\nRestrictions: Two provided datasets must have the same number of data and have same feature and name. For better use, it is recommended to use the news/texts that have same topics.\n\nHow to use: Place the datasets that consists of statements and feature values in a folder and put the name or path of the datasets in news1 and news2 arguments of multi_source_FE() definition.\n            news1 should be the dataset of referencing articles, and news2 should be the dataset of testing texts\n\nDate: Feb 2, 2020\nProgrammed by: Sasung Kim\n\n'

In [11]:
# Define the definitions for extracting features

def header_extension(data_header, extension, new_header):
    for i in data_header:
        new_header.append(f'{i} - {extension}')
    return new_header
    
    
# Subtract each features for each data in the same row (one-to-one) and save the data in total_list. List contains
# the extracted feature values related to texts. Ex. total_list[0] = all feature values of first statement

def one_to_one_dif(text, ref):
    feature_list = []
    feature = 0
    for i in range (len(text)):
        try:
            feature = (float(text[i]) - float(ref[i]))
        except:
            try:
                feature = (float(text[i]) - float(ref[i+2]))
            except:
                pass

        feature_list.append(round(feature, 2))
    return feature_list
    
    
def one_to_many_dif(text, ref):
    avg_list = []
    final_list = []
    for i in range(len(text)):
        for j in range(len(ref)):
            try:
                feature_val = float(text[i]) - float(ref[j])
            except:
                try:
                    feature_val = float(text[i]) - float(ref[i+2])
                except:
                    pass
            avg_list.append(feature_val)
        avg = round(mean(avg_list), 2)
        final_list.append(avg)
    return final_list
    
    
# Input: news1 (String), news2 (String), outFile (String), startFColumn (integer)
# Description: Read the data from news1 (reliable) and news2 (unreliable), and put the data in two lists. Then, subtract the news1 values in each column from news2 values.
#              The subtraction starts from startFColumn until the end of the .csv file. Next, the results will be saved in the file with given name or path from user (outFile)
#
# Return: New DataFrame ('MS_features')

def multi_source_FE(text, ref):

    # Saving original headers in header
    header = list(text)
    
    # Only feature values
    cols = [col for col in news_data_FE.columns if col not in ['Statement', 'Label']]
    
    # Assign data of the testing text and referencing text in data1 and data2
    if(text.equals(ref)):
        ref = text.shift(periods = 1)
        ref.iloc[0] = text.iloc[len(text)-1]
    
    state_text = text['Statement']
    state_ref = ref['Statement']
    label_text = text['Label']
    data_text = text[cols]
    data_ref = ref[cols]

    feature_num = len(data_text.keys())
    
    # Create header list with modified header names. Can be changed later with different method of multi-sourcing by modifying '- sub'.
    header_edit = ['text', 'news', 'label']
    for ele in cols:
        header_edit.append(ele)
    header_edit = header_extension(cols, 'sub', header_edit)
    header_edit = header_extension(cols, 'avg sub', header_edit)

    # Create local lists, dataframe that are used later in this definition.
    df = pd.DataFrame()
    sub = pd.DataFrame()
    avg_sub = pd.DataFrame()
    text = []
    ref = []
    total_list = []
    new_total_list = []
    
    # Loop through the data for subtraction
    for i in data_text.keys():
        sub[i + "- sub"] = one_to_one_dif(data_text[i], data_ref[i])
        avg_sub[i + "- avg sub"] = one_to_many_dif(data_text[i], data_ref[i])
    
    # Creating dataframe with multi-sourced features
    df['Text'] = state_text
    df['Reference'] = state_ref
    df['Label'] = label_text
    
    df1 = pd.concat([df, data_text, sub, avg_sub], axis = 1)
    df1 = df1.dropna(how = 'any')
    
    return df1

# Initiate the code and check for the dataframe
text_feature = multi_source_FE(text_data_FE, news_data_FE)
news_feature = multi_source_FE(news_data_FE, news_data_FE)
print(text_feature.head(5))
print(news_feature.head(5))

                                                Text  \
0  Coronaviruses caused SARS, MERS, and the curre...   
1  The coronavirus — which originated in Wuhan, C...   
2  The tech sector is feeling the impact of gover...   
3  Recommendations for blunting the spread of inf...   
4    The State Department issued the travel advisory   

                                           Reference  Label  # of Characters  \
0  Asians and Asian Americans are facing increase...    0.0              150   
1  Future cruisegoers are expressing concern over...    0.0              220   
2  US stock futures dropped Sunday evening as the...    0.0              115   
3  The novel coronavirus-related shock to US mark...    0.0              189   
4  Apple is temporarily closing all of its stores...    0.0               41   

   # of Words  # of Verbs  # of Noun  # of Sentence  \
0          28           7          9              3   
1          42           6         15              2   
2          19    

In [15]:
finalDF = pd.concat([text_feature, news_feature])
finalDF.to_csv("ActualNewsFeatureData.csv")
print(finalDF.head(5))


                                                Text  \
0  Coronaviruses caused SARS, MERS, and the curre...   
1  The coronavirus — which originated in Wuhan, C...   
2  The tech sector is feeling the impact of gover...   
3  Recommendations for blunting the spread of inf...   
4    The State Department issued the travel advisory   

                                           Reference  Label  # of Characters  \
0  Asians and Asian Americans are facing increase...    0.0            150.0   
1  Future cruisegoers are expressing concern over...    0.0            220.0   
2  US stock futures dropped Sunday evening as the...    0.0            115.0   
3  The novel coronavirus-related shock to US mark...    0.0            189.0   
4  Apple is temporarily closing all of its stores...    0.0             41.0   

   # of Words  # of Verbs  # of Noun  # of Sentence  \
0        28.0         7.0        9.0            3.0   
1        42.0         6.0       15.0            2.0   
2        19.0    

In [13]:
# Shuffle the data
finalDF = shuffle(finalDF)
print(finalDF)

                                                 Text  \
69  Wuhan coronavirus: Confirmed cases top 20,000 ...   
79  Fox News contributor Karl Rove breaks down the...   
39  Apple is no longer just closing one store in C...   
91  This week, President Trump's pardons, new Acti...   
24  Parents in McDonough, Georgia, were set to tra...   
..                                                ...   
22  As the coronavirus outbreak grows after killin...   
10  Factories in China and elsewhere are working a...   
46  British Airways has suspended all direct fligh...   
44  As Chinese authorities struggle to contain the...   
72  Passenger describes being stuck at sea amid co...   

                                            Reference  Label  # of Characters  \
69  Cruise ship outbreak worsens as China coronavi...    1.0            219.0   
79  Japan's economy is flirting with recession, an...    0.0             91.0   
39  Concerns are growing over the global spread of...    0.0            